In [38]:
import numpy as np
import numpy.random as nrand
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
import copy
from sklearn.linear_model import LinearRegression

In [3]:
with open('./FL_stratified/NK_landscape_stratified.pkl', 'rb') as f:
    NK_landscape_list = pickle.load(f)

In [16]:
with open('../index_file/epi_list_10s_all.pkl', 'rb') as f:
    epi_list = pickle.load(f)
with open('../index_file/pathway_list_10s_120000.pkl', 'rb') as f:
    pathway_list = pickle.load(f)

In [2]:
# Change the parameter to select different ruggedness measure.
metric = 'open_ratio' # 'N_max', 'epi', 'r_s', 'open_ratio'
N=10 # N=10 is the defalt value

In [49]:
def get_N_max(landscape):
    N_max = 0
    for gt in landscape:
        seq = gt[0:N]
        fit = gt[N]
        flag = True
        for i,_ in enumerate(seq):
            seq_ = copy.deepcopy(seq)
            seq_[i] = 1 - seq_[i]
            tmp = ''.join(seq_.astype(int).astype(str))
            idx = int(tmp, 2)
            fit_ = landscape[idx,N]
            if fit < fit_:
                flag = False
                break
        if flag == True:
            N_max += 1
    return N_max    

def cal_epi(landscape):
    epi_fit_list = landscape[epi_list][:,:,-1]
    n_epi = np.sum(np.sum(epi_fit_list[:,[0,0,3,3]] > epi_fit_list[:,[1,2,1,2]],axis=1)==4)
    return n_epi/len(epi_fit_list)

def cal_r_s(landscape):
    X = landscape[:,:N]
    y = landscape[:,-1]
    reg = LinearRegression().fit(X, y)
    y_predict = reg.predict(landscape[:,:N])
    roughness = np.sqrt(np.mean(np.square(y - y_predict)))
    slope = np.mean(np.abs(reg.coef_))
    return roughness/slope

def cal_open_ratio(landscape):
    pathway_fit_list = landscape[pathway_list][:,:,-1]
    total_open = np.sum(np.sum(pathway_fit_list[:,0:4]<=pathway_fit_list[:,1:5],axis=1)==pathway_fit_list.shape[1]-1)+\
    np.sum(np.sum(pathway_fit_list[:,0:4]<=pathway_fit_list[:,1:5],axis=1)==0)
    return total_open/len(pathway_list)
    
    
if metric == 'N_max':
    get_ruggedness = get_N_max

elif metric == 'r_s':
    get_ruggedness = cal_r_s

elif metric == 'epi':
    get_ruggedness = cal_epi

elif metric == 'open_ratio':
    get_ruggedness = cal_open_ratio
    
    

In [50]:
plot_dict = {'K':[],metric:[]}
for K in range(0,10):
    for j in range(100):
        plot_dict['K'].append(K)
        plot_dict[metric].append(get_ruggedness(NK_landscape_list[K][j]))
plot_df = pd.DataFrame(plot_dict)

In [51]:
# Uncomment only if you want to overwrite files in plot_df_data folder.
# with open(f'./plot_df_data/NK_{metric}_stratified_plot_df.pkl', 'wb') as f:
#     pickle.dump(plot_df,f)